In [ ]:
import pandas as pd
import re

In [339]:
# csv 불러오기
아편 = pd.read_csv('메스암페타민_concat(전체기간).csv')
메스암페타민 = pd.read_csv('아편_concat(전체기간).csv')

In [340]:
# 중복된 띄어쓰기,줄바꿈을 띄어쓰기 1개로 전체 바꾸기
아편['content'] = 아편['content'].str.replace('\s+', ' ')
메스암페타민['content'] = 메스암페타민['content'].str.replace('\s+', ' ')

<ipython-input-340-acb2097d46cd>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  아편['content'] = 아편['content'].str.replace('\s+', ' ')
<ipython-input-340-acb2097d46cd>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  메스암페타민['content'] = 메스암페타민['content'].str.replace('\s+', ' ')


In [341]:
#이모지 & 특수기호 제거
아편['content'] = 아편['content'].str.replace(pat = r'[^가-힣A-Za-z0-9.-/:\']', repl = r' ', regex = True)
메스암페타민['content'] = 메스암페타민['content'].str.replace(pat = r'[^가-힣A-Za-z0-9.-/:\']', repl = r' ', regex = True)

In [342]:
def clean_text(inputString):
  text_rmv = re.sub('ه ه [-=+,#\?^*\"※`~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]ܤه  ه ', ' ', inputString)
  text_rmv = ' '.join(text_rmv.split())
  return text_rmv

# 액센트 제거

def remove_accents(raw_text):
    raw_text = re.sub(u"[àáâãäåạ]", 'a', raw_text)
    raw_text = re.sub(u"[èéêë]", 'e', raw_text)
    raw_text = re.sub(u"[ìíîï]", 'i', raw_text)
    raw_text = re.sub(u"[òóôõö]", 'o', raw_text)
    raw_text = re.sub(u"[ùúûü]", 'u', raw_text)
    raw_text = re.sub(u"[ýÿ]", 'y', raw_text)
    raw_text = re.sub(u"[ß]", 'ss', raw_text)
    raw_text = re.sub(u"[ñ]", 'n', raw_text)
    raw_text = re.sub(u"[➍④]", '4', raw_text)
    raw_text = re.sub(u"𝒂𝒕𝒐", 'ato', raw_text)
    raw_text = re.sub(u"Аҳŗịṅḡ", 'axring', raw_text)
    raw_text = re.sub(u"аҳŗịṅḡ", 'axring', raw_text)
    return raw_text

In [343]:
for i in range(len(아편['content'])):
    a = clean_text(아편['content'][i])
    a = remove_accents(a)
    아편['content'][i] = a

<ipython-input-343-6b79531e4ae8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  아편['content'][i] = a


In [344]:
for i in range(len(메스암페타민['content'])):
    a = clean_text(메스암페타민['content'][i])
    a = remove_accents(a)
    메스암페타민['content'][i] = a

<ipython-input-344-5ad589363d7b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  메스암페타민['content'][i] = a


In [345]:
# 추가 전처리
#punct = "'?!,#$%\'()*+-/;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', } 
searchfilter_mapping = {"ㅌ ㄹ ㄱ ㄹ": "텔레그램", "ㅌ ㄹ ㄱ ㄹ": "텔레그램","ㅌㄹ": "텔레", "ㅌ ㄹ": "텔레", "ㅌㄹㄱㄹ": "텔레그램", "텔 레": "텔레", "그 램":"그램", "텔레 그램":"텔레그램","문 의":"문의","ka톡":"카톡", "ka 톡": "카톡", "ｋa톡":"카톡", "카 카 오 톡":"카카오톡", "gt":"", "lt":"", "p i n k m a n 1 1 4": "pinkman114", "F a s t i c e119": "Fastice119"}

def clean(text, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    # for p in punct:
    #     text = text.replace(p, f' {p} ')

    # for p in punct:
    #     text = text.replace(p, f' {p} ') 
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': '', 'ه ':'', '∞': ''}
    for s in specials:
        text = text.replace(s, specials[s])

    return text.strip()

def clean_str(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(r'http\S+', repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = re.compile(r"(?<=\b\w)\s(?=\w\b)") # 띄어쓰기 된  아이디 결합
    text = re.sub(pattern=pattern, repl='', string=text)
    #pattern = '[^\w\s\n]'         # 특수기호제거
    #text = re.sub(pattern=pattern, repl='', string=text)
    #text = re.sub('[-=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', string=text)
    #text = re.sub('\n', '.', string=text)
    return text 

In [346]:
for i in range(len(아편['content'])):
    a = clean(아편['content'][i], punct_mapping)
    a = clean(아편['content'][i], searchfilter_mapping)
    a = clean_str(a)
    아편['content'][i] = a

for i in range(len(메스암페타민['content'])):
    a = clean(메스암페타민['content'][i], punct_mapping)
    a = clean(메스암페타민['content'][i], searchfilter_mapping)
    a = clean_str(a)
    메스암페타민['content'][i] = a

<ipython-input-346-8d571b1a9184>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  아편['content'][i] = a
<ipython-input-346-8d571b1a9184>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  메스암페타민['content'][i] = a


In [347]:
# 중복된 띄어쓰기,줄바꿈을 띄어쓰기 1개로 전체 바꾸기
아편['content'] = 아편['content'].str.replace('\s+', ' ')
메스암페타민['content'] = 메스암페타민['content'].str.replace('\s+', ' ')

<ipython-input-347-acb2097d46cd>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  아편['content'] = 아편['content'].str.replace('\s+', ' ')
<ipython-input-347-acb2097d46cd>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  메스암페타민['content'] = 메스암페타민['content'].str.replace('\s+', ' ')


In [348]:
for text in range(len(아편['content'])):
  print(아편['content'][text])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
뽕라덴 복귀 텔레 binladen11 전국 실시간 좌표 전국 드랍 샘플 20만원 시작 아이스작대기 후리하는방법 아이스사끼 작대기아이스 엘에디 아이스뽕 케이 최음제 떨팝니다 캔디 작대기 엑시 빙두 떨팝니다 엑시 아이스작대기 
뽕라덴 복귀 텔레 binladen11 전국 실시간 좌표 전국 드랍 샘플 20만원 시작 아이스작대기 후리하는방법 아이스사끼 작대기아이스 엘에디 아이스뽕 케이 최음제 떨팝니다 캔디 작대기 엑시 빙두 떨팝니다 엑시 아이스작대기 
텔레 문의 Happy bbong 해피 해피뽕 아이스작대기 전문 딜러 필로폰파는곳 마약효과 삥두 후리하는방법 브액판매처 김치 떨후기 인디카 사탕 케타민 떨파는곳 빙두 작대기팝니다 차가운술 떨 아이스작대기 아이스작대기 
텔레 문의 Happy bbong 해피 해피뽕 아이스작대기 전문 딜러 필로폰파는곳 마약효과 삥두 후리하는방법 브액판매처 김치 떨후기 인디카 사탕 케타민 떨파는곳 빙두 작대기팝니다 차가운술 떨 아이스작대기 아이스작대기 
텔레 문의 Happy bbong 해피 해피뽕 아이스작대기 전문 딜러 차가운술파는곳 강간데이트 아이스게이 강간데이트 인잭하는법 빠른구매 케타민 떨느낌 떨판매 떨사는곳 빙두 작대기팝니다 아이스술 LSD 아이스술 아이스작대기 
텔레 문의 Happy bbong 해피 해피뽕 아이스작대기 전문 딜러 차가운술파는곳 강간데이트 아이스게이 강간데이트 인잭하는법 빠른구매 케타민 떨느낌 떨판매 떨사는곳 빙두 작대기팝니다 아이스술 LSD 아이스술 아이스작대기 
Happy bbong 해피 해피뽕 아이스작대기 전문 딜러 아이스거래방법 마약효과 차가운술파는곳 빙두판매처 게이만남 작대기사용법 인디카 떨구매 대마 대마 케타민 허브액상 샘플20만 대마 아이스팝니다 아이스술 아이스작대기 
Happy bbong 해피 해피뽕 아이스작대기 전문 딜러 아이스거래방법 마약효과 차가운술파는곳 빙두판매처 게이만남 작대기사용법 인디카 떨구매 대마 대마 케타민 허브액상 샘플20만 대

In [352]:
아편.to_csv('아편_fianl.csv', index = False)

In [353]:
메스암페타민.to_csv('메스암페타민_final.csv', index = False)